In [ ]:
# Install Libraries

# mount Google Drive
from google.colab import drive
drive.mount('/content/colabIntern')
%cd "/content/colabIntern/MyDrive/2023/Langara/Capstone Project/Collab"
#%cd "/content/colabIntern/MyDrive/Capstone/Capstone Project/Collab"

Mounted at /content/colabIntern
/content/colabIntern/.shortcut-targets-by-id/1lwwFhOdnZo3Bx0mzGW8l1qPio2d8s1TV/Langara/Capstone Project/Collab


In [ ]:
time.___version__

NameError: ignored

In [ ]:
globals().clear
import time
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
from datetime import datetime

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Load Dataset
#df = pd.read_csv('../../df.csv')
df = pd.read_csv('df.csv')

In [ ]:
# list of tickers for stocks in our data set. Sort the ticker list since our df will be alphabetically arranged.
tickers = ["AAPL", 'XOM', 'IBM', 'KO', 'CVX', 'BA', 'PFE', 'MSFT', 'T', 'WMT',
       'F', 'NFLX', 'JPM', 'MCD', 'GE', 'NVDA', 'JNJ', 'BAC', 'C', 'AMZN',
       'INTC', 'CSCO', 'TSLA', 'GOOGL', 'AMD', 'BABA', 'VZ', 'DIS',
       'META']
tickers.sort()

In [ ]:
# For this specific trial drop PLTR since data is not complete
df = df[df.stock_ID != "PLTR"]

In [ ]:
df['DATETIME']= pd.to_datetime(df['DATETIME'], format='%m/%d/%Y %H:%M')

In [ ]:
# Sort by time so that the first 29 rows occupy the first time value for all the stocks.
# Fill in 0's for missing values for now.
df = df.set_index('DATETIME')
df = df.fillna(0)

In [ ]:
# Select the length of the df ; For this file we need 7 months
# We use the first 3 months to make the first prediction, then shift window
# Then we will repeat this for the next 4 months (Hence total is 7 months needed)
df = df.sort_index().loc['2022-06-01':'2022-12-30']

In [ ]:
#Set the DATETIME for fecha
DATETIME = df.index.values

In [ ]:
# Sort dataframe such that it is both in sequential order, and also in alphabetical order for each day
# (i.e first entry for each time entry should be AAPL, and last should be XOM).
df.sort_values(["DATETIME", "stock_ID"], inplace=True)

In [ ]:
df.head()

,stock_ID,Close,Volume_PP,US2Y_PP,US5Y_PP,US10Y_PP,DJ_PP,NQ_PP,SP_PP,Gold_PP,...,Sector_Financial Services,Sector_Healthcare,Sector_Industrial,Sector_Technology,P/E Ratio ttm,P/B Ratio mrq,D/E Ratio mrq,Free Cash Flow ttm,PEG Ratio 5 year expected,ROE ttm
DATETIME,,,,,,,,,,,,,,,,,,,,,
2022-06-01 09:30:00,AAPL,150.73,8129985.0,1.1,2.84,2.85,33081.01,12229.75,4152.96,1843.3,...,0,0,0,1,27.96,45.93,195.87,84.73,2.78,147.94
2022-06-01 09:30:00,AMD,102.20,4904039.0,1.1,2.84,2.85,33081.01,12229.75,4152.96,1843.3,...,0,0,0,1,110.08,2.72,5.40,3.79,1.81,4.24
2022-06-01 09:30:00,AMZN,124.00,10810540.0,1.1,2.84,2.85,33081.01,12229.75,4152.96,1843.3,...,0,0,0,0,0.00,7.16,116.36,7.62,2.30,-1.92
2022-06-01 09:30:00,BA,130.78,176460.0,1.1,2.84,2.85,33081.01,12229.75,4152.96,1843.3,...,0,0,1,0,0.00,0.00,0.00,5.08,6.53,0.00
2022-06-01 09:30:00,BABA,96.04,232511.0,1.1,2.84,2.85,33081.01,12229.75,4152.96,1843.3,...,0,0,0,0,56.28,1.87,17.37,41.83,2.10,2.28


In [ ]:
df=df.iloc[:,:74]

In [ ]:
file_name = 'XGB.60d.1d'
num_companies = 29  # 29 companies in our dataset.
step_rows  = 24 * num_companies  # 24 time periods per day per stock
total_rows = len(df['2022-06-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(df['2022-06-01':'2022-08-31'])  # Define length of training window

scale_X = MinMaxScaler()
model = RandomForestRegressor(n_estimators = 100, max_depth = 100, n_jobs=-1)

result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = df.iloc[i:train_rows + i, 1:]
    test   = df.iloc[train_rows + i:train_rows + i + step_rows, 1:]
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]



    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model.fit(X_train, y_train)
    y_hat = model.predict(X_test)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    result = pd.concat([result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Count Down: 81
Count Down: 80
Count Down: 79
Count Down: 78
Count Down: 77
Count Down: 76
Count Down: 75
Count Down: 74
Count Down: 73
Count Down: 72
Count Down: 71
Count Down: 70
Count Down: 69
Count Down: 68
Count Down: 67
Count Down: 66
Count Down: 65
Count Down: 64
Count Down: 63
Count Down: 62
Count Down: 61
Count Down: 60
Count Down: 59
Count Down: 58
Count Down: 57
Count Down: 56
Count Down: 55
Count Down: 54
Count Down: 53
Count Down: 52
Count Down: 51
Count Down: 50
Count Down: 49
Count Down: 48
Count Down: 47
Count Down: 46
Count Down: 45
Count Down: 44
Count Down: 43
Count Down: 42
Count Down: 41
Count Down: 40
Count Down: 39
Count Down: 38
Count Down: 37
Count Down: 36
Count Down: 35
Count Down: 34
Count Down: 33
Count Down: 32
Count Down: 31
Count Down: 30
Count Down: 29
Count Down: 28
Count Down: 27
Count Down: 26
Count Down: 25
Count Down: 24
Count Down: 23
Count Down: 22
Count Down: 21
Count Down: 20
Count Down: 19
Count Down: 18
Count Down: 17
Count Down: 16
Count Down

In [ ]:
metrics_df = pd.DataFrame(columns = ['stock_ID', "RMSE", "MAPE", "MPE", "MTT"])

In [ ]:
tsla= result[25::29]

In [ ]:
def mean_positive_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    error = np.mean(np.maximum((y_pred - y_true),0))
    return error

In [ ]:
new_rows = []
for i in range(len(tickers)):
    stock_result = result[i::29]
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
    # Calculate MAPE %
    mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
    # Calculate MPE
    mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
    # Calculate MTT in seconds
    mtt = np.mean(stock_result['TRAIN_DURATION'])
    new_row = pd.Series([tickers[i],rmse, mape, mpe, mtt], index=metrics_df.columns)
    new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)

In [ ]:
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,AAPL,38.214669,22.892197,0.562265,39.872286
1,AMD,49.818472,67.322917,45.056178,39.872286
2,AMZN,24.274872,19.083627,14.013136,39.872286
3,BA,33.835959,16.385375,4.640689,39.872286
4,BABA,39.970207,43.379435,33.405808,39.872286
5,BAC,78.052878,222.800939,75.491134,39.872286
6,C,73.551843,154.664548,70.708041,39.872286
7,CSCO,74.082668,158.999969,71.437535,39.872286
8,CVX,38.513253,19.186750,1.002987,39.872286
9,DIS,32.961842,27.298022,25.683016,39.872286


In [ ]:
avg_values=metrics_df.mean()
avg_df = pd.DataFrame(avg_values).T
avg_df